In [35]:
# Importación de los datos
import pandas as pd
import sqlite3

## 1. CARGAR PEDIDOS (CSV)

In [36]:
# Cargar el fichero pedidos.csv

df_csv = pd.read_csv("C:/Users/ruben/analisis-datos/pedidos.csv", encoding="utf-16", sep="|", na_values=["NA", "null", "-", "###ERROR###", "-99999.00", "-99999.0"])
df_csv.columns = df_csv.columns.str.strip()

## 2. CARGAR CLIENTES (JSON)


In [37]:
# Cargar el fichero clientes.json
df_json = pd.read_json("C:/Users/ruben/analisis-datos/clientes.json")


## 3. CARGAR PRODUCTOS (SQLite)


In [38]:
# Cargar el fichero productos.db

with sqlite3.connect("C:/Users/ruben/analisis-datos/productos.db") as conn:
    df_bd = pd.read_sql_query("SELECT * FROM productos;", conn)


## 4. NORMALIZAR TIPOS DE CLAVE

In [39]:
# Convertimos los valores de las columnas que actuarán de clave a str para evitar conflictos
df_csv["id_cliente"] = df_csv["id_cliente"].astype(str)
df_csv["id_producto"] = df_csv["id_producto"].astype(str) 
df_csv["Id Pedido"] = df_csv["Id Pedido"].astype(str)

df_json["id_cliente"] = df_json["id_cliente"].astype(str)

df_bd["id_producto"] = df_bd["id_producto"].astype(str)

## 5. UNIÓN DE DATASETS

In [40]:
# Unir los tres datasets en uno solo
df_merged = df_csv.merge(df_json, on='id_cliente', how='left')
df_merged_global = df_merged.merge(df_bd, on='id_producto', how='left')


## Apartado 1: Información básica del dataset


In [41]:
# 1.1 Mostrar las primeras 5 filas del dataset global
print(df_merged_global.head(5))

# 1.2 Mostrar las dimensiones del dataset (filas y columnas)
print(df_merged_global.ndim)

# 1.3 Mostrar los nombres de todas las columnas
print(df_merged_global.columns)

# 1.4 Mostrar información sobre los tipos de datos de las columnas
print(df_merged_global.dtypes)

# 1.5 Mostrar las últimas 3 filas del dataset
print(df_merged_global.tail(3))


  Id Pedido id_cliente id_producto fecha_pedido  cantidad  precio_unitario  \
0       1.0     1263.0       531.0   2025-02-28       7.0           356.14   
1       nan        nan       744.0   2023-11-18       8.0          1569.27   
2       3.0     3060.0       990.0   2023-10-18      10.0           707.74   
3       4.0     1216.0       174.0   2024-05-31       3.0          1281.67   
4       5.0     1558.0       916.0   2024-06-30       9.0          1208.76   

      total    metodo_pago estado_pedido        País envío  ... genero edad  \
0   2492.98        Tarjeta     Pendiente           Namibia  ...    NaN  NaN   
1  12554.16  Transferencia     Pendiente  Christmas Island  ...    NaN  NaN   
2   7077.40         PayPal     Pendiente          Tanzania  ...    NaN  NaN   
3   3845.01   Criptomoneda     Pendiente          Bulgaria  ...    NaN  NaN   
4  10878.84         PayPal       Enviado           Romania  ...    NaN  NaN   

  nivel_fidelizacion nombre_producto    categoria precio

## Apartado 2: Tipos de datos y valores nulos


In [42]:
# 2.1 Mostrar información detallada del DataFrame (tipos, memoria, nulos)
print(df_merged_global.info())


# 2.2 Contar valores nulos por columna

print(df_merged_global.isnull().sum())

# 2.3 Mostrar solo las columnas que tienen valores nulos
print(df_merged_global[df_merged_global.columns[df_merged_global.isnull().any()]])

# 2.4 Calcular el porcentaje de valores nulos por columna
porcentaje_nulos = (df_merged_global.isnull().mean()*100)
print(porcentaje_nulos)

# 2.5 Verificar si hay filas completamente vacías
print(df_merged_global[df_merged_global.isnull().all(axis=1)])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id Pedido           40000 non-null  object 
 1   id_cliente          40000 non-null  object 
 2   id_producto         40000 non-null  object 
 3   fecha_pedido        38728 non-null  object 
 4   cantidad            38700 non-null  float64
 5   precio_unitario     38637 non-null  float64
 6   total               38689 non-null  float64
 7   metodo_pago         38710 non-null  object 
 8   estado_pedido       38692 non-null  object 
 9   País envío          38670 non-null  object 
 10  nombre              0 non-null      object 
 11  email               0 non-null      object 
 12  telefono            0 non-null      object 
 13  direccion           0 non-null      object 
 14  fecha_registro      0 non-null      object 
 15  genero              0 non-null      object 
 16  edad

## Apartado 3: Estadísticas descriptivas


In [43]:
# 3.1 Estadísticas descriptivas de todas las columnas numéricas
print(df_merged_global.describe())

# 3.2 Estadísticas descriptivas de todas las columnas (numéricas y categóricas)
print(df_merged_global.describe(include='all'))

# 3.3 Calcular la media, mediana y moda de la columna 'total'
print(df_merged_global['total'].mean())
print(df_merged_global['total'].median())
print(df_merged_global['total'].mode())

# 3.4 Calcular el rango (máximo - mínimo) de la columna 'cantidad'
print(df_merged_global['cantidad'].max() - df_merged_global['cantidad'].min())

# 3.5 Calcular la desviación estándar y la varianza de la columna 'total'
print(df_merged_global['total'].std(axis=0))
print(df_merged_global['total'].var())

# 3.6 Contar valores únicos en columnas categóricas

print(df_merged_global.select_dtypes(include='object').nunique())


           cantidad  precio_unitario         total  edad
count  38700.000000     38637.000000  38689.000000   0.0
mean       5.493359       981.587831   5411.297785   NaN
std        2.871951       587.671591   4623.478111   NaN
min        1.000000         1.000000      1.000000   NaN
25%        3.000000       468.680000   1618.200000   NaN
50%        5.000000       979.460000   4047.750000   NaN
75%        8.000000      1494.940000   8220.780000   NaN
max       10.000000      2000.000000  19994.400000   NaN
       Id Pedido id_cliente id_producto fecha_pedido      cantidad  \
count      40000      40000       40000        38728  38700.000000   
unique     38053       4998        1001          733           NaN   
top          nan        nan         nan           20           NaN   
freq        1320       1341        1320          666           NaN   
mean         NaN        NaN         NaN          NaN      5.493359   
std          NaN        NaN         NaN          NaN      2.871951 

## Apartado 4: Filtrado simple

In [44]:
# 4.1 Filtrar pedidos con cantidad mayor a 5 unidades
print(df_merged_global[df_merged_global['cantidad']>5])

# 4.2 Filtrar pedidos con total mayor a 1000 euros
print(df_merged_global[df_merged_global['total']>1000])

# 4.3 Filtrar pedidos con precio unitario menor a 50 euros
print(df_merged_global[df_merged_global['precio_unitario']<50])

# 4.4 Filtrar pedidos con cantidad igual a 1
print(df_merged_global[df_merged_global['cantidad']==1])

# 4.5 Mostrar los primeros 5 pedidos grandes (cantidad > 5)
print(df_merged_global[df_merged_global['cantidad']>5].head(5))

# 4.6 Mostrar los primeros 5 pedidos caros (total > 1000€)
print(df_merged_global[df_merged_global['total']>1000].head(5))

      Id Pedido id_cliente id_producto fecha_pedido  cantidad  \
0           1.0     1263.0       531.0   2025-02-28       7.0   
1           nan        nan       744.0   2023-11-18       8.0   
2           3.0     3060.0       990.0   2023-10-18      10.0   
4           5.0     1558.0       916.0   2024-06-30       9.0   
5           6.0     2354.0       373.0   2023-10-08       8.0   
...         ...        ...         ...          ...       ...   
39990   39991.0     4968.0       528.0   2025-02-16      10.0   
39991   39992.0      727.0       194.0   2024-12-20       8.0   
39992   39993.0     1306.0       422.0   2024-01-06       6.0   
39997   39998.0     4163.0       196.0   2025-01-26       6.0   
39999   40000.0     2417.0       284.0           20       6.0   

       precio_unitario     total    metodo_pago estado_pedido  \
0               356.14   2492.98        Tarjeta     Pendiente   
1              1569.27  12554.16  Transferencia     Pendiente   
2               707.74  

## Apartado 5: Filtrado con múltiples condiciones

In [45]:
# 5.1 Filtrar pedidos enviados con cantidad mayor a 3
df_merged_global[(df_merged_global['cantidad']>3) & (df_merged_global['estado_pedido']=='Enviado')]

# 5.2 Filtrar pedidos de tarjeta o PayPal con total mayor a 500 euros
df_merged_global[(df_merged_global['metodo_pago']=='Tarjeta') | (df_merged_global['metodo_pago']=='PayPal')]


# 5.3 Filtrar pedidos pendientes o cancelados con cantidad menor a 2
df_merged_global[(df_merged_global['estado_pedido']=='Cancelado') | (df_merged_global['cantidad']<2)]
# 5.4 Filtrar pedidos enviados o devueltos con precio unitario mayor a 100
df_merged_global[(df_merged_global['estado_pedido'].isin(['Devuelto','Cancelado'])) | (df_merged_global['precio_unitario']>100)]

# 5.5 Mostrar los primeros 5 pedidos enviados grandes
print(df_merged_global[(df_merged_global['cantidad']>5) & (df_merged_global['estado_pedido']=='Enviado')].head(5))


# 5.6 Mostrar los primeros 5 pedidos premium
print(df_merged_global[df_merged_global['nivel_fidelizacion']=='Oro'].head(5))



   Id Pedido id_cliente id_producto fecha_pedido  cantidad  precio_unitario  \
4        5.0     1558.0       916.0   2024-06-30       9.0          1208.76   
20      21.0     2225.0       407.0   2024-04-29       8.0           555.18   
21      22.0     3978.0       686.0          NaN       6.0           462.08   
24      25.0     1878.0       585.0   2025-07-18       8.0          1315.42   
26      27.0     2609.0       258.0   2024-10-07      10.0           150.08   

       total    metodo_pago estado_pedido  \
4   10878.84         PayPal       Enviado   
20   4441.44         PayPal       Enviado   
21   2772.48  Transferencia       Enviado   
24  10523.36   Criptomoneda       Enviado   
26   1500.80         PayPal       Enviado   

                                      País envío  ... genero edad  \
4                                        Romania  ...    NaN  NaN   
20                                       Bahamas  ...    NaN  NaN   
21                                    Uzbekista

## Apartado 6: Filtrado temporal

In [46]:

# 6.1 Convertir la columna fecha_pedido a tipo datetime
df_merged_global['fecha_pedido'] = pd.to_datetime(df_merged_global['fecha_pedido'], dayfirst=False, errors='coerce')

# 6.2 Filtrar pedidos del año 2024
df_merged_global[df_merged_global['fecha_pedido'].dt.year == 2024]

# 6.3 Filtrar pedidos del mes de enero de 2024
df_merged_global[df_merged_global['fecha_pedido'].dt.month == 1]


# 6.4 Filtrar pedidos del primer trimestre de 2024
df_merged_global[(df_merged_global['fecha_pedido']>='2024-01-01') & (df_merged_global['fecha_pedido']<='2024-03-01')]

# 6.5 Filtrar pedidos de los lunes (día de la semana = 0)
print(df_merged_global[df_merged_global['fecha_pedido'].dt.day_of_week == 0])

# 6.6 Mostrar los primeros 5 pedidos de 2024
df_merged_global[(df_merged_global['fecha_pedido']>='2024-01-01') & (df_merged_global['fecha_pedido']<='2024-12-31')].head(5)


      Id Pedido id_cliente id_producto fecha_pedido  cantidad  \
10         11.0      317.0       102.0   2024-06-17       1.0   
20         21.0     2225.0       407.0   2024-04-29       8.0   
25         26.0     2571.0       749.0   2024-03-18       5.0   
26         27.0     2609.0       258.0   2024-10-07      10.0   
31         32.0      494.0       489.0   2025-08-04       1.0   
...         ...        ...         ...          ...       ...   
39949   39950.0      801.0       867.0   2025-08-04       7.0   
39965   39966.0     4771.0       190.0   2024-07-01       7.0   
39975   39976.0     2126.0       637.0   2024-07-22       6.0   
39986   39987.0     2401.0       793.0   2023-11-20       1.0   
39993   39994.0     1626.0       895.0   2024-08-26       4.0   

       precio_unitario     total    metodo_pago estado_pedido  \
10              985.33    985.33         PayPal      Devuelto   
20              555.18   4441.44         PayPal       Enviado   
25             1582.55  

,Id Pedido,id_cliente,id_producto,fecha_pedido,cantidad,precio_unitario,total,metodo_pago,estado_pedido,País envío,...,genero,edad,nivel_fidelizacion,nombre_producto,categoria,precio_base,descuento,stock,proveedor,fecha_alta
3,4.0,1216.0,174.0,2024-05-31,3.0,1281.67,3845.01,Criptomoneda,Pendiente,Bulgaria,...,NaN,NaN,NaN,Only Max,Electrónica,348.97,20.0,324.0,"Ramsey, Anderson and Rivera",2025-03-29
4,5.0,1558.0,916.0,2024-06-30,9.0,1208.76,10878.84,PayPal,Enviado,Romania,...,NaN,NaN,NaN,Own X,Deporte,823.87,10.0,404.0,"Alvarado, Rodriguez and Wu",2023-04-05
7,8.0,285.0,481.0,2024-12-17,7.0,187.82,1314.74,Tarjeta,Devuelto,Antarctica (the territory South of 60 deg S),...,NaN,NaN,NaN,Let Pro,Libros,384.89,0.0,224.0,"Jimenez, Knight and King",2023-08-01
8,9.0,374.0,524.0,2024-09-25,3.0,781.23,2343.69,Criptomoneda,Enviado,Mayotte,...,NaN,NaN,NaN,Article Pro,Juguetes,1951.3,5.0,303.0,"Barrett, Spears and Larson",2022-12-17
10,11.0,317.0,102.0,2024-06-17,1.0,985.33,985.33,PayPal,Devuelto,Kyrgyz Republic,...,NaN,NaN,NaN,Own Max,Deporte,1740.23,20.0,355.0,"Cruz, Stout and Smith",2022-12-06


## Apartado 7: Estado del pedido

In [ ]:
# 7.1 Contar el número de pedidos por cada estado
df_merged_global.groupby('estado_pedido').size()

# 7.2 Calcular el porcentaje de pedidos por estado
print(df_merged_global['estado_pedido'].value_counts(normalize=True)*100)

# 7.3 Calcular el total de ventas por estado
print(df_merged_global.groupby('estado_pedido')['total'].sum())

# 7.4 Calcular el valor promedio de pedidos por estado
print(df_merged_global.groupby('estado_pedido')['total'].mean())

# 7.5 Calcular la cantidad total de productos vendidos por estado
print(df_merged_global.groupby('estado_pedido')['cantidad'].sum())

# 7.6 Mostrar estadísticas completas por estado
print(df_merged_global.groupby('estado_pedido').agg(
    cantidad_pedidos = ('total','count'),
    total_ventas = ('cantidad','sum'),
    media_ventas = ('cantidad', 'mean')
))


Series([], Name: proportion, dtype: float64)
estado_pedido
##              22051.05
Ca             984724.44
Cancelado    50191215.62
De             925505.48
Devuelto     49753601.27
En             826068.69
Enviado      48321220.31
Pe             885745.99
Pendiente    50732206.15
Name: total, dtype: float64
estado_pedido
##           3150.150000
Ca           5896.553533
Cancelado    5433.713935
De           5199.468989
Devuelto     5402.128260
En           5399.141765
Enviado      5360.685635
Pe           5335.819217
Pendiente    5458.010344
Name: total, dtype: float64
estado_pedido
##              30.0
Ca             920.0
Cancelado    50651.0
De             904.0
Devuelto     50897.0
En             828.0
Enviado      49582.0
Pe             836.0
Pendiente    51191.0
Name: cantidad, dtype: float64


## Apartado 8: Método de pago

In [ ]:
# 8.1 Contar el número de pedidos por método de pago
df_merged_global.groupby('metodo_pago').size()

# 8.2 Calcular el porcentaje de uso de cada método de pago
print(df_merged_global['metodo_pago'].value_counts(normalize=True)*100)


# 8.3 Calcular el valor total de ventas por método de pago
print(df_merged_global.groupby('metodo_pago')['total'].sum())


# 8.4 Calcular el valor promedio de pedidos por método de pago
print(df_merged_global.groupby('metodo_pago')['total'].mean())

# 8.5 Calcular la cantidad promedio de productos por método de pago
print(df_merged_global.groupby('estado_pedido')['cantidad'].mean())

# 8.6 Mostrar estadísticas completas por método de pago (numero de pedidos, total de ventas, promedio de pedidos, cantidad total, promedio de la cantidad)
print(df_merged_global.groupby('metodo_pago').agg(
    num_pedidos = ('total','count'),
    total_ventas = ('total','sum'),
    promedio_pedidos = ('total', 'mean'),
    cantidad_total = ('cantidad', 'sum'),
    promedio_cantidad = ('cantidad','mean')
))


metodo_pago
Tarjeta          24.784293
Criptomoneda     24.781710
Transferencia    24.500129
PayPal           24.381297
Cr                0.410747
Pa                0.387497
Tr                0.382330
Ta                0.359080
##                0.012917
Name: proportion, dtype: float64
metodo_pago
##                  27252.42
Cr                 755778.35
Criptomoneda     49735344.31
Pa                 757002.32
PayPal           50108987.00
Ta                 800707.50
Tarjeta          49939798.46
Tr                 816526.50
Transferencia    49689229.21
Name: total, dtype: float64
metodo_pago
##               5450.484000
Cr               4907.651623
Criptomoneda     5367.509638
Pa               5331.002254
PayPal           5478.188149
Ta               5931.166667
Tarjeta          5384.344847
Tr               5790.968085
Transferencia    5407.468627
Name: total, dtype: float64
estado_pedido
##           4.285714
Ca           5.750000
Cancelado    5.477560
De           5.349112
Devuelto

## Apartado 9: País de envío

In [ ]:
# 9.1 Contar el número total de países únicos
df_merged_global['País envío'].nunique()

# 9.2 Mostrar los top 10 países por número de pedidos
print(df_merged_global.groupby('País envío')['Id Pedido'].count().sort_values(ascending=False).head(10))

# 9.3 Mostrar los top 10 países por valor total de ventas
print(df_merged_global.groupby('País envío')['total'].sum().sort_values(ascending=False).head(10))


# 9.4 Calcular el valor promedio de pedidos por país (top 10)
print(df_merged_global.groupby('País envío')['total'].mean().sort_values(ascending=False).head(10))


# 9.5 Calcular la cantidad total de productos enviados por país (top 10)
print(df_merged_global[df_merged_global['estado_pedido'] == 'Enviado'].groupby('País envío')['cantidad'].sum().sort_values(ascending=False).head(10))


# 9.6 Mostrar estadísticas del país con más pedidos (número de pedidos, total de ventas, promedio de pedido y total de cantidad)
print(df_merged_global.groupby('País envío').agg(
    num_pedidos = ('total','count'),
    total_ventas = ('total','sum'),
    promedio_pedidos = ('total', 'mean'),
    cantidad_total = ('cantidad', 'sum')
))




353

## Apartado 10: Análisis demográfico

In [ ]:
# 10.1 Contar el número total de clientes únicos
print(df_merged_global['id_cliente'].nunique())

# 10.2 Distribución de clientes por género
print(df_merged_global.groupby('genero')['id_cliente'].count())

# 10.3 Porcentaje de distribución por género
print(df_merged_global['genero'].value_counts(normalize=True) * 100)

# 10.4 Distribución de clientes por nivel de fidelización
print(df_merged_global.groupby('nivel_fidelizacion')['id_cliente'].count())

# 10.5 Estadísticas básicas de edad
print(f"Suma: {df_merged_global['edad'].sum()}")
print(f"Media: {df_merged_global['edad'].mean()}")
print(f"Mediana: {df_merged_global['edad'].median()}")
print(f"Desviación estándar: {df_merged_global['edad'].std()}")
print(f"Varianza: {df_merged_global['edad'].var()}")
print(f"Mínimo: {df_merged_global['edad'].min()}")
print(f"Máximo: {df_merged_global['edad'].max()}")

# 10.6 Mostrar estadísticas completas de edad por género (número de clientes, edad promedio, edad mínima, edad máxima y mediana de edad)
print(df_merged_global.groupby('genero').agg(
    num_clientes = ('id_cliente','count'),
    edad_promedio = ('edad','mean'),
    edad_min = ('edad', 'min'),
    edad_max = ('edad', 'max'),
    edad_mediana = ('edad', 'median')
))



## Apartado 11: Clientes por país

In [ ]:
# 11.1 Contar el número total de países únicos donde viven los clientes


# 11.2 Mostrar los top 15 países con más clientes


# 11.3 Calcular el porcentaje de clientes por país (top 10)


# 11.4 Mostrar estadísticas (número de clientes, edad promedio, edad mínima, edad máxima y mediana de edad) de edad por país (top 5 países)


# 11.5 Mostrar distribución de género por país (top 3 países)


# 11.6 Mostrar distribución de nivel de fidelización por país (top 3 países)




## Apartado 12: Clientes por ciudad

In [ ]:
# 12.1 Mostrar las top 20 ciudades con más clientes


# 12.2 Mostrar las ciudades con exactamente 1 cliente


# 12.3 Mostrar las ciudades con más de 10 clientes




## Apartado 13: Categoría de productos

In [ ]:
# 13.1 Mostrar las categorías cuyo nombre tiene más de 6 caracteres y que tienen entre 50 y 160 productos


# 13.2 Mostrar las categorías que contienen la letra "o" (mayúscula o minúscula) y tienen menos de 120 productos



## Apartado 14: Precios

In [ ]:
# 14.1 Mostrar estadísticas básicas de precios base (precio promedio, precio mínimo, precio máximo y mediana)


# 14.2 Calcular el precio promedio por categoría


# 14.3 Encontrar los productos más caros y más baratos


# 14.4 Mostrar los top 10 productos más caros




## Apartado 15: Productos más vendidos

In [ ]:

# 15.1 Obtener el producto más vendido (cantidad vendida, total vendido, nombre del producto y categoría)



## Apartado 16: Análisis temporal


In [ ]:
# 16.1 Obtener información sobre las ventas por mes (número total de ventas, número de pedidos y cantidad total)


# 16.2 Obten la misma información pero en lugar de por mes, por semana.



## Apartado 17: Duplicados

In [ ]:
## 17.1 Contar filas duplicadas en cada uno de los datasets originales.



## Apartado 18: Valores faltantes

In [ ]:
# 18.1 Obtener el porcentaje de valores faltantes en cada columna del dataset.


## Apartado 19: Valoración final

Tras completar este ejercicio, ¿qué conclusiones has obtenido acerca de los datos? ¿Consideras que sería necesario aplicar algún tipo de preprocesamiento o crees que los datos son adecuados tal como están? Apoya tu valoración con ejemplos concretos de columnas que ilustren tu análisis.

<!-- Responde aquí al apartado 19 -->
**Responder**

